In [1]:
import pandas
from SentiCR.SentiCR import SentiCR

SentiCR/data/so-train.pkl


In [21]:
df_original = pandas.read_xml('test.xml')

In [22]:
df = df_original[4:].drop(['team_domain', 'channel_name', 'start_date', 'end_date'], axis=1)

In [23]:
df[df['conversation_id'] == 4.0]

,conversation_id,ts,user,text
20,4.0,2020-08-01T02:00:57.155000,Josselin,probably
21,4.0,2020-08-01T02:00:57.606000,Josselin,why
22,4.0,2020-08-01T02:23:42.043000,Samarie,I think you probably can with uintptr and with...
23,4.0,2020-08-01T02:23:54.182000,Samarie,then go would probably not think twice about t...
24,4.0,2020-08-01T02:23:56.283000,Samarie,but I'm not sure
...,...,...,...,...
102,4.0,2020-08-01T04:15:27.152000,Cephas,My apologies i don't think i mentioned use of ...
104,4.0,2020-08-01T04:18:06.108000,Samarie,i just didn't think of it
105,4.0,2020-08-01T04:23:23.296000,Cephas,😛
106,4.0,2020-08-01T04:23:31.234000,Cephas,Thank you for always being active and answer o...


In [24]:
response_df = df['conversation_id'].value_counts().rename_axis('conversation_id').to_frame('nr_messages').sort_index(ascending=True)
response_df

,nr_messages
conversation_id,
1.0,7
2.0,7
3.0,2
4.0,85
5.0,1
...,...
1911.0,1
1912.0,4
1913.0,12


In [25]:
first_message_df = df.sort_values(by=['ts']).groupby("conversation_id").first()
count_and_text_df = response_df.join(first_message_df).rename(columns={"text": "first_message", "user": "user_first_message", "ts": "ts_first_message"})

In [26]:
count_and_text_df

,nr_messages,ts_first_message,user_first_message,first_message
conversation_id,,,,
1.0,7,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works
2.0,7,2020-08-01T01:09:01.202000,Reeves,Hwllo
3.0,2,2020-08-01T01:27:32.834000,Treysen,The built in proxy stuff is nice
4.0,85,2020-08-01T02:00:57.155000,Josselin,probably
5.0,1,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...
...,...,...,...,...
1911.0,1,2020-09-30T14:22:11.133000,Zechariah,Hi all the following code is to create a thumb...
1912.0,4,2020-09-30T15:03:49.551000,Samarie,is this a HTTP request?
1913.0,12,2020-09-30T15:50:10.240000,Mingliang,https://golang.org/src/net/http/request.go#L812


In [27]:
sentiment_analyzer=SentiCR()

StackOverflow Reading data from oracle..
Training classifier model..


/home/tom/.pyenv/versions/3.7.15/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ani', 'continu', 'deleg', 'doe', 'doubl', 'dure', 'els', 'endwhil', 'extend', 'implement', 'includ', 'interfac', 'namespac', 'nativ', 'nowwhil', 'onc', 'ourselv', 'overrid', 'packag', 'privat', 'protect', 'rais', 'readon', 'requir', 'sign', 'synchron', 'themselv', 'tri', 'veri', 'yourselv'] not in stop_words.
  % sorted(inconsistent)


Training used 8.59 seconds


In [28]:
import numpy as np
from sklearn.metrics import classification_report
import time
def predict_sentiCR(df):
    begin=time.time()
    sentences=df['first_message']
    pred=[]
    for sent in sentences:
        score=sentiment_analyzer.get_sentiment_polarity(sent)
        pred.append(score)
    
    end=time.time()
    print('Prediction used {:.2f} seconds'.format(end-begin))
    return pred

In [29]:
predictions = predict_sentiCR(count_and_text_df)

Prediction used 3.61 seconds


In [31]:
def Extract(lst):
    return [item[0] for item in lst]
df_with_sentiment = count_and_text_df.copy()
df_with_sentiment['sentiCR'] = Extract(predictions)


In [32]:
df_with_sentiment

,nr_messages,ts_first_message,user_first_message,first_message,sentiCR
conversation_id,,,,,
1.0,7,2020-08-01T00:01:15.595000,Samarie,{{ with .Data }} also works,1
2.0,7,2020-08-01T01:09:01.202000,Reeves,Hwllo,0
3.0,2,2020-08-01T01:27:32.834000,Treysen,The built in proxy stuff is nice,0
4.0,85,2020-08-01T02:00:57.155000,Josselin,probably,0
5.0,1,2020-08-01T03:17:19.489000,Maiana,i've codegened > 600 go files that all wrap in...,0
...,...,...,...,...,...
1911.0,1,2020-09-30T14:22:11.133000,Zechariah,Hi all the following code is to create a thumb...,2
1912.0,4,2020-09-30T15:03:49.551000,Samarie,is this a HTTP request?,0
1913.0,12,2020-09-30T15:50:10.240000,Mingliang,https://golang.org/src/net/http/request.go#L812,0


In [34]:
# df_with_sentiment.to_csv('sentiCR.csv')

In [47]:
# df_with_sentiment['nr_messages_normalized'] = df_with_sentiment['nr_messages'] /df_with_sentiment['nr_messages'].abs().max()
corr=df_with_sentiment['nr_messages'].corr(df_with_sentiment['sentiCR'])

In [48]:
corr

0.003499373952088837